In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from IPython.core.interactiveshell import InteractiveShell
import statsmodels.api as sm
import statsmodels.api as sm
InteractiveShell.ast_node_interactivity = "all"

    1. value weight portfolio return 구하기
        -> sum(eretadj * me_lag1) / sum(me_lag1)
    2. BM 5분위 - BM 1분위 (excess return 계산)
    3. Size, BM 순으로 dependent sort함
    4. fama french three factor (mktrf, smb, hml)를 독립변수로 두고, y를 BM5-BM1 excess return으로 두고 regression함
    5. 알파와 t통계 위주로 보기
        -> alpha가 합쳐서 평균적으로 0이 되기때문에 fama french로 설명 가능하다는 것.
        -> 이 때 size, BM dependent sort했기 때문에, size는 control 되는 것임

In [12]:
# SAS 3: Calculate the time-series average of value-weighted portfolio returns
#portfolio1 정리
portfolio1 = pd.read_csv('./assignment2_data.csv')
# % term으로 바꾸고, 한달 전 ME을 곱하여줌
portfolio1['eretadj_ME']=(portfolio1['eretadj']*100)*portfolio1['ME_lag1']
portfolio1=portfolio1.sort_values(by=['date','p1','p2'])
portfolio1.reset_index(inplace=True)
#나중에 eretadj는 t+1시점의 값으로 적용해야하므로 column을 따로 만들어줌

In [13]:
#portfolio2 만들기 - value weighted portfolio
portfolio2=pd.DataFrame()
# a: eretadj에 me_lag1을 곱하고 모두 더하여줌
portfolio2['eretadj_ME_Sum']=portfolio1.groupby(['date','p1','p2']).apply(lambda d:(d.eretadj*d.ME_lag1*100).sum())
# b; me_lag1을 sum하기
portfolio2['ME_lag_Sum']=portfolio1.groupby(['date','p1','p2'])['ME_lag1'].sum()
# a / b 값을 통해 weight형태로 만들어줌 
portfolio2['vm_pret']=portfolio2['eretadj_ME_Sum']/portfolio2['ME_lag_Sum']
portfolio2.pop('eretadj_ME_Sum')
portfolio2.pop('ME_lag_Sum')

date      p1  p2
19630228  1   1    -5.804558e+02
              2     4.638412e+00
              3    -4.494788e+02
              4     5.329962e+02
              5    -2.915269e+02
                        ...     
20121231  5   1    -5.821863e+06
              2     1.271966e+06
              3     1.587222e+05
              4     2.619257e+06
              5     8.493734e+06
Name: eretadj_ME_Sum, Length: 14975, dtype: float64

date      p1  p2
19630228  1   1     2.189385e+02
              2     2.644112e+02
              3     2.496469e+02
              4     2.537011e+02
              5     1.079780e+02
                        ...     
20121231  5   1     2.373180e+06
              2     2.418550e+06
              3     1.935223e+06
              4     2.174046e+06
              5     1.950381e+06
Name: ME_lag_Sum, Length: 14975, dtype: float64

In [14]:
# Calculate the return difference between the fifth (p2 = 5) and first (p2 = 1) BM sorted portfolios within each Size sorted portfolio;
# portfolio3 만들기 -> p2가 1 or 5인 것만 남기기
portfolio3 = portfolio2.copy().unstack()
portfolio3.pop(('vm_pret',2))
portfolio3.pop(('vm_pret',3))
portfolio3.pop(('vm_pret',4))
portfolio3 = portfolio3.stack()

date      p1
19630228  1     0.017542
          2    -2.519387
          3     0.649269
          4    -0.877091
          5    -3.020763
                  ...   
20121231  1     4.195254
          2     3.353203
          3     3.968558
          4     2.474137
          5     0.525921
Name: (vm_pret, 2), Length: 2995, dtype: float64

date      p1
19630228  1    -1.800458
          2    -1.309487
          3    -0.518167
          4     0.197494
          5    -2.921324
                  ...   
20121231  1     4.399218
          2     3.818418
          3     2.318688
          4     2.059264
          5     0.082017
Name: (vm_pret, 3), Length: 2995, dtype: float64

date      p1
19630228  1     2.100882
          2     0.006890
          3    -2.302808
          4    -1.405281
          5    -3.162041
                  ...   
20121231  1     4.374290
          2     4.111120
          3     1.751740
          4     3.551877
          5     1.204784
Name: (vm_pret, 4), Length: 2995, dtype: float64

In [15]:
#portfolio4 만들기 -> (_5)-(_1)
portfolio4=portfolio3.copy().unstack()
portfolio4['VM_pret']=portfolio4['vm_pret'][5]-portfolio4['vm_pret'][1]
portfolio4.pop(('vm_pret',1))
portfolio4.pop(('vm_pret',5))
portfolio4=portfolio4.stack()
portfolio4=portfolio4.reset_index()
portfolio4.pop('p2')
portfolio4['p2']=51
portfolio4.rename(columns={'VM_pret':'vm_pret'},inplace=True)
#portfolio5 만들기 -> portfolio2 & portfolio4 concat
portfolio4=portfolio4.set_index(['date','p1','p2'])
portfolio5=pd.concat([portfolio2,portfolio4],axis=0,join='inner').sort_index()
portfolio5=portfolio5.reset_index()
portfolio5['date_temp']=pd.to_datetime(portfolio5['date'].map(lambda x : str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:8]))
portfolio5['year']=portfolio5['date_temp'].dt.year
portfolio5['month']=portfolio5['date_temp'].dt.month
portfolio5.pop('date_temp')
portfolio5=portfolio5.sort_values(by=['year','month','date','p1','p2'])

date      p1
19630228  1    -2.651228
          2    -0.251242
          3    -5.105673
          4     0.494442
          5    -1.612517
                  ...   
20121231  1     1.256330
          2     2.709436
          3     2.470460
          4     1.950883
          5    -2.453191
Name: (vm_pret, 1), Length: 2995, dtype: float64

date      p1
19630228  1    -2.699874
          2     1.892853
          3    -0.008091
          4    -4.612406
          5    -3.548749
                  ...   
20121231  1     4.707813
          2     4.105095
          3     3.576346
          4     3.971480
          5     4.354911
Name: (vm_pret, 5), Length: 2995, dtype: float64

0        
1        
2        
3        
4        
       ..
2990     
2991     
2992     
2993     
2994     
Name: p2, Length: 2995, dtype: object

0       1963-02-28
1       1963-02-28
2       1963-02-28
3       1963-02-28
4       1963-02-28
           ...    
17965   2012-12-31
17966   2012-12-31
17967   2012-12-31
17968   2012-12-31
17969   2012-12-31
Name: date_temp, Length: 17970, dtype: datetime64[ns]

In [17]:
factors

,year,month,mktrf,smb,hml
0,1926,7,2.96,-2.30,-2.87
1,1926,8,2.64,-1.40,4.19
2,1926,9,0.36,-1.32,0.01
3,1926,10,-3.24,0.04,0.51
4,1926,11,2.53,-0.20,-0.35
...,...,...,...,...,...
1127,2020,6,2.46,2.70,-2.22
1128,2020,7,5.77,-2.18,-1.31
1129,2020,8,7.63,-0.26,-2.95
1130,2020,9,-3.63,0.06,-2.56


In [18]:
portfolio6

,date,p1,p2,vm_pret,year,month,mktrf,smb,hml
0,19630228,1,1,-2.651228,1963,2,-2.38,0.50,2.21
1,19630228,1,2,0.017542,1963,2,-2.38,0.50,2.21
2,19630228,1,3,-1.800458,1963,2,-2.38,0.50,2.21
3,19630228,1,4,2.100882,1963,2,-2.38,0.50,2.21
4,19630228,1,5,-2.699874,1963,2,-2.38,0.50,2.21
...,...,...,...,...,...,...,...,...,...
17965,20121231,5,2,0.525921,2012,12,1.18,1.45,3.52
17966,20121231,5,3,0.082017,2012,12,1.18,1.45,3.52
17967,20121231,5,4,1.204784,2012,12,1.18,1.45,3.52
17968,20121231,5,5,4.354911,2012,12,1.18,1.45,3.52


In [16]:
# convert factors from decimal to percent
factors = pd.read_csv('./factors_monthly.csv')
factors['mktrf']=factors['mktrf']*100
factors['smb']=factors['smb']*100
factors['hml']=factors['hml']*100
factors['date']=pd.to_datetime(factors['date'].map(lambda x : str(x)[:4] + '-' + str(x)[4:6] + '-' +str(x)[6:8]))
factors['year']=factors['date'].dt.year
factors['month']=factors['date'].dt.month
factors=factors[['year','month','mktrf','smb','hml']]
portfolio6 = pd.merge(portfolio5,factors,on=['year','month'],how='left')

In [19]:
df = portfolio6[portfolio6["p2"] == 51]
df_1 = df[df["p1"] == 1]
df_2 = df[df["p1"] == 2]
df_3 = df[df["p1"] == 3]
df_4 = df[df["p1"] == 4]
df_5 = df[df["p1"] == 5]

In [15]:
Y = df_1["vm_pret"]
X = df_1[["mktrf", "smb", "hml"]]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
re1 = results.get_robustcov_results(cov_type='HAC', maxlags = 7)
re1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                vm_pret   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.589
Method:                 Least Squares   F-statistic:                     104.3
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           2.98e-54
Time:                        00:20:37   Log-Likelihood:                -1466.4
No. Observations:                 599   AIC:                             2941.
Df Residuals:                     595   BIC:                             2958.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6550      0.138      4.733      0.000       0.383       0.927
mktrf         -0.1136      0.042     -2.682      0.008      -0.197      -0.030
smb           -0.2334      0.079     -2.966      0.003      -0.388      -0.079
hml            1.0287      0.070     14.670      0.000       0.891       1.166
==============================================================================
Omnibus:                       31.360   Durbin-Watson:                   1.821
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               73.477
Skew:                          -0.257   Prob(JB):                     1.11e-16
Kurtosis:                       4.637   Cond. No.                         4.91
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 7 lags and without small sample correction
"""

In [16]:
Y = df_2["vm_pret"]
X = df_2[["mktrf", "smb", "hml"]]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
re2 = results.get_robustcov_results(cov_type='HAC', maxlags = 7)
re2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                vm_pret   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.699
Method:                 Least Squares   F-statistic:                     290.4
Date:                Wed, 09 Dec 2020   Prob (F-statistic):          4.42e-116
Time:                        00:20:37   Log-Likelihood:                -1392.4
No. Observations:                 599   AIC:                             2793.
Df Residuals:                     595   BIC:                             2810.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2836      0.112      2.535      0.011       0.064       0.503
mktrf         -0.1195      0.028     -4.324      0.000      -0.174      -0.065
smb           -0.1529      0.055     -2.769      0.006      -0.261      -0.044
hml            1.2120      0.045     26.769      0.000       1.123       1.301
==============================================================================
Omnibus:                        3.921   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.141   Jarque-Bera (JB):                4.708
Skew:                          -0.003   Prob(JB):                       0.0950
Kurtosis:                       3.434   Cond. No.                         4.91
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 7 lags and without small sample correction
"""

In [17]:
Y = df_3["vm_pret"]
X = df_3[["mktrf", "smb", "hml"]]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
re3 = results.get_robustcov_results(cov_type='HAC', maxlags = 7)
re3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                vm_pret   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     102.1
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           2.55e-53
Time:                        00:20:37   Log-Likelihood:                -1404.2
No. Observations:                 599   AIC:                             2816.
Df Residuals:                     595   BIC:                             2834.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1187      0.111      1.070      0.285      -0.099       0.337
mktrf         -0.1394      0.039     -3.553      0.000      -0.216      -0.062
smb           -0.2834      0.082     -3.440      0.001      -0.445      -0.122
hml            1.3153      0.092     14.231      0.000       1.134       1.497
==============================================================================
Omnibus:                       38.174   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              143.891
Skew:                          -0.051   Prob(JB):                     5.68e-32
Kurtosis:                       5.399   Cond. No.                         4.91
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 7 lags and without small sample correction
"""

In [18]:
Y = df_4["vm_pret"]
X = df_4[["mktrf", "smb", "hml"]]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
re4 = results.get_robustcov_results(cov_type='HAC', maxlags = 7)
re4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                vm_pret   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     83.68
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           3.54e-45
Time:                        00:20:37   Log-Likelihood:                -1424.8
No. Observations:                 599   AIC:                             2858.
Df Residuals:                     595   BIC:                             2875.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2351      0.124     -1.903      0.058      -0.478       0.008
mktrf         -0.0755      0.035     -2.128      0.034      -0.145      -0.006
smb           -0.1808      0.102     -1.770      0.077      -0.381       0.020
hml            1.2797      0.091     14.052      0.000       1.101       1.459
==============================================================================
Omnibus:                       94.256   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              395.092
Skew:                          -0.645   Prob(JB):                     1.61e-86
Kurtosis:                       6.764   Cond. No.                         4.91
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 7 lags and without small sample correction
"""

In [19]:
Y = df_5["vm_pret"]
X = df_5[["mktrf", "smb", "hml"]]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
re5 = results.get_robustcov_results(cov_type='HAC', maxlags = 7)
re5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                vm_pret   R-squared:                       0.642
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     65.08
Date:                Wed, 09 Dec 2020   Prob (F-statistic):           2.11e-36
Time:                        00:20:37   Log-Likelihood:                -1409.6
No. Observations:                 599   AIC:                             2827.
Df Residuals:                     595   BIC:                             2845.
Df Model:                           3                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4049      0.112     -3.629      0.000      -0.624      -0.186
mktrf          0.0039      0.037      0.106      0.916      -0.068       0.076
smb            0.0736      0.052      1.424      0.155      -0.028       0.175
hml            1.2147      0.089     13.715      0.000       1.041       1.389
==============================================================================
Omnibus:                      110.621   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              701.803
Skew:                           0.639   Prob(JB):                    4.03e-153
Kurtosis:                       8.147   Cond. No.                         4.91
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 7 lags and without small sample correction
"""